Name:        EU grid population

Purpose:     The purpose of this script is to calculate density of OpenStreetMap (OSM) road/railway network of chosen area per area and per capita.
It calculates two indicators:
rd/rlw_density (roads/railways length in km per 1 km2 of area),
rd/rlw_per_capita (roads/railways length in m per 1 inhabitant).

Author:      Adam Tóth

This script is a part of bachelor thesis "Possibilities of calculation characteristics of the transport
network of states and cities"
supervisor: doc. Ing. Zdena Dobešová, Ph.D.
Department of Geoinformatics, Faculty of Science, Palacký University in Olomouc, Czech republic

Created:     21.03.2022

In [1]:
import arcpy
arcpy.env.overwriteOutput = True
print("The script has started!")

The script has started!


Set the geodatabase as a workspace.

In [5]:
workspace = "D:/transport_netw_char/transport_netw_char.gdb/"
arcpy.env.workspace = workspace


Variables ```data``` and ```area``` store the roads and area of interest layers names.

In [6]:
data = "osm_roads_clipped"
area = "main_FI003L4_TURKU_UA2018_Boundary"

In [7]:
pop_data = "D:/3.ZS/bakalarka/EU_POP_GRID_2018/JRC_POPULATION_2018.shp"

# hex_or_own = "false"
# own_layer = ""

size = "10 SquareKilometers"
cor_sys_string = 'PROJCS["ETRS_1989_LAEA",GEOGCS["GCS_ETRS_1989",DATUM["D_ETRS_1989",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Azimuthal_Equal_Area"],PARAMETER["False_Easting",4321000.0],PARAMETER["False_Northing",3210000.0],PARAMETER["Central_Meridian",10.0],PARAMETER["Latitude_Of_Origin",52.0],UNIT["Meter",1.0]]'

Select the population grid "pixels" that intersect with the area of interest. Copy or export these pixels into your workspace and then don't forget to clear the selection.

In [10]:
control_selection = arcpy.management.SelectLayerByLocation(pop_data, "INTERSECT", area)
arcpy.management.CopyFeatures(control_selection, "pop_data_copy")
control_selection = arcpy.management.SelectLayerByAttribute(pop_data, "CLEAR_SELECTION")


Save the "pixel" area size in a new field which will be named ```"area_orig"```. You will make a copy of the ```"Shape_Area"``` field basically.

In [11]:
arcpy.management.AddField("pop_data_copy", "area_orig", "DOUBLE")
arcpy.management.CalculateField("pop_data_copy", "area_orig", '!Shape_Area!')

<Result 'D:/transport_netw_char/transport_netw_char.gdb\\pop_data_copy'>

Clip the copied/exported population grid "pixels" by the area of interest.

In [12]:
arcpy.analysis.Clip("pop_data_copy", area, "clipped_pop_data")
pop_data = "clipped_pop_data"

Add a new field named ```"P_2018_orig"``` and populate it with the current population.

In [14]:
arcpy.management.AddField(pop_data, "P_2018_orig", "DOUBLE")
arcpy.management.CalculateField(pop_data, "P_2018_orig", '(!TOT_P_2018!/!area_orig!)*!Shape_Area!')

<Result 'D:/transport_netw_char/transport_netw_char.gdb\\clipped_pop_data'>

Generate a hexagonal grid of hexagon size 10 km over the area of interest and then clip it by its polygon layer.

In [15]:
arcpy.management.GenerateTessellation("hex_grid", area, "HEXAGON", size)
arcpy.analysis.Clip("hex_grid", area, "hex_gr")

<Result 'D:/transport_netw_char/transport_netw_char.gdb\\hex_gr'>

Field ```"area_orig"``` will contain the area size of current clipped "pixels".

In [16]:
arcpy.management.CalculateField(pop_data, "area_orig", '!Shape_Area!')

<Result 'D:/transport_netw_char/transport_netw_char.gdb\\clipped_pop_data'>

Now you'll get the population information into the hexagons. Intersect first the population grid by the hexagonal grid.

In [17]:
arcpy.analysis.Intersect([pop_data, "hex_gr"], "pop_data_isect", "ALL")

<Result 'D:/transport_netw_char/transport_netw_char.gdb\\pop_data_isect'>

Add field into the intersected population grid's attribute table and calculate there population size using the same principle as it was used when calculating population after clipping.

In [18]:
arcpy.management.AddField("pop_data_isect", "new_pop2018", "DOUBLE")
arcpy.management.CalculateField("pop_data_isect", "new_pop2018", '(!P_2018_orig!/!area_orig!)*!Shape_Area!')

<Result 'D:/transport_netw_char/transport_netw_char.gdb\\pop_data_isect'>

In [19]:
arcpy.management.Dissolve("pop_data_isect", "pop_data_isect_diss", "FID_hex_gr", [["new_pop2018","SUM"]])

<Result 'D:/transport_netw_char/transport_netw_char.gdb\\pop_data_isect_diss'>

In [ ]:
print("part 5: preparing and cutting roads layer")

selected_features = arcpy.management.SelectLayerByAttribute(data, "NEW_SELECTION", "code > 5110 And code < 5136")
arcpy.conversion.FeatureClassToFeatureClass(selected_features, workspace, "lines_export")
selected_features = arcpy.management.SelectLayerByAttribute(data, "CLEAR_SELECTION")

arcpy.analysis.Intersect(["lines_export", "hex_gr"], "rd_isect", "ONLY_FID")
arcpy.management.Dissolve("rd_isect", "rd_isect_diss", "FID_hex_gr")

print("Roads were cut and dissolved by hexagonal grid")


In [ ]:
print("part 6: adding information about population and road length to the hexagonal grid")

arcpy.management.JoinField("hex_gr", "OBJECTID", "pop_data_isect_diss", "FID_hex_gr", ["SUM_new_pop2018"])

arcpy.management.AddField("rd_isect_diss", "rd_length", "DOUBLE")
arcpy.management.CalculateField("rd_isect_diss", "rd_length", '!Shape_Length!')
arcpy.management.JoinField("hex_gr", "OBJECTID", "rd_isect_diss", "FID_hex_gr", ["rd_length"])

print("Join successful")

In [ ]:
print("part 7: calculating roads density per capita and per square km")

arcpy.management.AddFields("hex_gr", [["rd_density", "DOUBLE"], ["rd_per_capita", "DOUBLE"]])
print("fields added")

arcpy.management.CalculateField("hex_gr", "rd_density", '(!rd_length!/1000)/(!Shape_Area!/1000000)')
print("areal density done")

arcpy.management.CalculateField("hex_gr", "rd_per_capita", '!rd_length!/!SUM_new_pop2018!')
print("population density done")

print("Roads density calculated")